<a href="https://colab.research.google.com/github/mdzikrim/DeepLearning/blob/main/CNN_Model_SVHN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
import numpy as np

# Setup device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [2]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = datasets.SVHN(root='./data', split='train', download=True, transform=transform)
test_dataset = datasets.SVHN(root='./data', split='test', download=True, transform=transform)

train_len = int(0.8 * len(train_dataset))
val_len = len(train_dataset) - train_len
train_data, val_data = random_split(train_dataset, [train_len, val_len])

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
val_loader = DataLoader(val_data, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)


100%|██████████| 182M/182M [00:03<00:00, 49.6MB/s]


100%|██████████| 64.3M/64.3M [00:02<00:00, 28.0MB/s]


In [3]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.25)
        self.fc1 = nn.Linear(64 * 8 * 8, 256)
        self.fc2 = nn.Linear(256, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 32x16x16
        x = self.pool(F.relu(self.conv2(x)))  # 64x8x8
        x = x.view(-1, 64 * 8 * 8)
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [11]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        running_loss = 0
        correct, total = 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        train_acc = correct / total * 100
        print(f"Epoch {epoch+1}, Loss: {running_loss:.4f}, Training Accuracy: {train_acc:.2f}%")

# Evaluation function
def evaluate_model(model, loader):
    model.eval()
    y_true = []
    y_pred = []
    y_probs = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            outputs = model(images)
            probs = F.softmax(outputs, dim=1)
            predictions = torch.argmax(probs, dim=1)

            y_true.extend(labels.numpy())
            y_pred.extend(predictions.cpu().numpy())
            y_probs.extend(probs.cpu().numpy())

    # Metrics
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    y_probs = np.array(y_probs)

    acc = accuracy_score(y_true, y_pred)
    prec = precision_score(y_true, y_pred, average='macro')
    rec = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    try:
        auc = roc_auc_score(y_true, y_probs, multi_class='ovr')
    except:
        auc = None

    print(f"Accuracy: {acc*100:.2f}%")
    print(f"Precision: {prec:.4f}")
    print(f"Recall: {rec:.4f}")
    print(f"F1-Score: {f1:.4f}")
    print(f"AUC (ROC): {auc:.4f}" if auc else "AUC (ROC): Not available")

# Inisialisasi dan jalankan
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_model(model, train_loader, val_loader, criterion, optimizer, epochs=10)

print("\nEvaluation on Validation Set:")
evaluate_model(model, val_loader)

print("\nEvaluation on Test Set:")
evaluate_model(model, test_loader)


Epoch 1, Loss: 803.3928, Training Accuracy: 72.48%
Epoch 2, Loss: 429.2291, Training Accuracy: 86.09%
Epoch 3, Loss: 348.7404, Training Accuracy: 88.56%
Epoch 4, Loss: 296.2694, Training Accuracy: 90.16%
Epoch 5, Loss: 258.3306, Training Accuracy: 91.39%
Epoch 6, Loss: 228.9688, Training Accuracy: 92.37%
Epoch 7, Loss: 203.0803, Training Accuracy: 93.29%
Epoch 8, Loss: 180.7150, Training Accuracy: 93.85%
Epoch 9, Loss: 159.4059, Training Accuracy: 94.57%
Epoch 10, Loss: 142.8343, Training Accuracy: 95.14%

Evaluation on Validation Set:
Accuracy: 90.44%
Precision: 0.8988
Recall: 0.8981
F1-Score: 0.8984
AUC (ROC): 0.9918

Evaluation on Test Set:
Accuracy: 89.85%
Precision: 0.8894
Recall: 0.8899
F1-Score: 0.8891
AUC (ROC): 0.9908
